# Liberaries

In [59]:
import glob
import pandas as pd
import os

In [60]:
folder_path = r"D:\_Computer_Science_\DEPI\The Final Project"
years_str=range(2019, 2024)

# Removing Headers

In [61]:
# function to remove the metadata from everyearfile
def remove_first_n_lines(file_path, n=18):
    lines = []
    with open(file_path, "r") as f:
        lines = f.readlines()[n:]
    with open(file_path, "w") as f:
        f.writelines(lines)

In [62]:
for file_path in glob.glob(f"{folder_path}/data_monthly_original/**/*.csv", recursive=True):
    remove_first_n_lines(file_path)

# Append

## append the common in all the years

In [63]:
cols = set()
for file_name in glob.glob(f"{folder_path}/data_monthly_original/*/*.csv", root_dir=folder_path, recursive=True):
    cols.add(file_name.split('\\')[-1].split('.')[0])
cols = list(cols)
print(cols)
print(len(cols))

['ALLSKY_SFC_SW_DWN', 'ALLSKY_SFC_UV_INDEX', 'PW', 'GWETPROF', 'WS50M', 'T2M', 'ALLSKY_SFC_UVA', 'Z0M', 'WD50M', 'TS_MAX', 'T10M_MAX', 'T2M_MAX', 'TSOIL2', 'EVLAND', 'TO3', 'CLRSKY_DAYS', 'WS2M_MAX', 'GWETTOP', 'TSOIL6', 'TSOIL3', 'T10M_MIN', 'RHOA', 'WS2M_MIN', 'WD2M', 'WS50M_MIN', 'RH2M', 'CLOUD_AMT', 'CLOUD_AMT_NIGHT', 'PRECSNO', 'WS2M', 'PRECTOTCORR', 'MIDDAY_INSOL', 'TS', 'TSOIL5', 'GWETROOT', 'QV2M', 'TSOIL4', 'T2M_MIN', 'CLOUD_AMT_DAY', 'TS_MIN', 'WS50M_MAX', 'AIRMASS', 'PSH', 'TSOIL1', 'ALLSKY_SFC_UVB', 'T10M']
46


In [64]:
for col in cols:
    dataframes = []
    for file_path in glob.glob(f"{folder_path}/data_monthly_original/*/{col}.csv", root_dir=folder_path, recursive=True):
        dataframes.append(pd.read_csv(file_path))
    df = pd.concat(dataframes)
    df.to_csv(f"{folder_path}/data_monthly_appended/{col}.csv", index=False)

# Melt

In [66]:
for file_name in glob.glob(f"{folder_path}/data_monthly_appended/*.csv", root_dir=folder_path, recursive=True):
    df = pd.read_csv(file_name)
    col = df['PARAMETER'][0]
    df = pd.melt(df.drop(['PARAMETER', 'ANN'], axis=1), id_vars=['YEAR', 'LAT', 'LON'],
                 value_vars=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'],
                 var_name='MONTH', value_name=col)
    df.to_csv(f"{folder_path}/data_monthly_melt/{col}.csv", index=False)

# Merging files

In [71]:
def Merge():
    df_small=None
    df_large=None
    for cur_file in glob.glob(f"{folder_path}/data_monthly_melt/*.csv"):
        test=pd.read_csv(cur_file).reset_index(drop=True)
        
        if df_small is None and (len(test)==33672):
            df_small=test
        elif df_large is None and (len(test)==151776):
            df_large=test
        else :
            if df_small is not None and (len(test)==33672):
                df_small = pd.merge(df_small, test, on=['YEAR', 'LAT', 'LON', 'MONTH'], how='outer').reset_index(drop=True)
            elif df_large is not None and (len(test)==151776):
                df_large = pd.merge(df_large, test, on=['YEAR', 'LAT', 'LON', 'MONTH'], how='outer').reset_index(drop=True)

    if df_small is not None:
        df_small.drop_duplicates().to_csv(f"{folder_path}/data_monthly_merged/df_small.csv", index=False)
    if df_large is not None:
        df_large.drop_duplicates().to_csv(f"{folder_path}/data_monthly_merged/df_large.csv", index=False)


In [72]:
Merge()